In [74]:
import findspark
findspark.init()

In [75]:
from pyspark import SparkConf, SparkContext

In [76]:
import codecs

In [ ]:
conf= SparkConf().setMaster("local").setAppName("PopularMovies")
sc= SparkContext(conf =conf)

In [80]:
def loadMovieNames():
    movieNames = {}
    # The following line is updated from what's in the video to handle
    # encoding issues on some Ubuntu systems:
    with codecs.open("ml-100k/u.ITEM", "r", encoding='utf-8', errors='ignore') as f:
        for line in f:
            fields = line.split('|')
            movieNames[int(fields[0])] = fields[1]
    return movieNames

In [81]:
nameDict = sc.broadcast(loadMovieNames())

In [82]:
lines=sc.textFile('ml-100k/u.data')

In [83]:
lines.take(5)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596']

In [17]:
movies=lines.map(lambda x:(x.split()[1],1))

In [84]:
movies.take(5)

[('242', 1), ('302', 1), ('377', 1), ('51', 1), ('346', 1)]

In [85]:
moviecounts=movies.reduceByKey(lambda x,y:x+y)

In [86]:
moviecounts.take(5)

[('242', 117), ('302', 297), ('377', 13), ('51', 81), ('346', 126)]

In [87]:
flipped = moviecounts.map( lambda x : (x[1], x[0]))

In [88]:
flipped.take(5)

[(117, '242'), (297, '302'), (13, '377'), (81, '51'), (126, '346')]

In [89]:
sortedMovies = flipped.sortByKey()

In [98]:
sortedMoviesWithNames = sortedMovies.map(lambda countMovie : (nameDict.value[countMovie[1]], countMovie[0]))

In [ ]:
sortedMoviesWithNames.take(5)

In [54]:
nameDict.value[1348]

'Every Other Weekend (1990)'

In [27]:
nameDict.value[2]

'GoldenEye (1995)'

In [ ]:
results = sortedMoviesWithNames.collect()

In [ ]:
for result in results:
    print (result)